Training and comparing the peformance of Linear, Lasso(L1 regularization) and Ridge(L2 regularization) Regressions using the diabetes dataset.
Performance metrics: MSE, R2 score.

In [251]:
from sklearn.datasets import load_iris, load_diabetes
from sklearn.model_selection import KFold, cross_val_score
import pandas as pd
import numpy as np
# irisDataset = load_iris()

# print("Keys in the iris dataset are: ")
# for key in irisDataset.keys():
#     print(key, end="\n")

# X = pd.DataFrame(irisDataset.data)
# y = pd.DataFrame(irisDataset.target)
# # print(X.shape)
# # print(y.shape)

#######################################Code continues below####################


In [252]:
diabetesData = load_diabetes()  #load diabetes dataset to the diabetesData variable
for key in diabetesData.keys(): #Assessment of keys of dataset
    print(key, end="\n")

# print(diabetesData.DESCR, end="\n")
#Constructing the data array(predictor) into a pandas dataframe
X = pd.DataFrame(diabetesData.data, columns=diabetesData.feature_names)

X

data
target
frame
DESCR
feature_names
data_filename
target_filename
data_module


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [253]:
#Constructing the target array(response/target) into a pandas dataframe
y = pd.DataFrame(diabetesData.target, columns=["DiseaseProgression"], )
y
# print(diabetesData.DESCR)

,DiseaseProgression
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0
...,...
437,178.0
438,104.0
439,132.0
440,220.0


In [254]:
# #Splitting the dataset
# from sklearn.model_selection import train_test_split as ttsplit

# XTrain, XTest, yTrain, yTest = ttsplit(diabetesPredictorDataFrame, diabetesTargetDataFrame)

#Here, we import our metrics:
from sklearn.metrics import mean_squared_error, r2_score

#Next, we create numpy 2D array to hold our calculated mse and r2 score metrics for 
# the train and test sets. We wil use it later to visualize their performances tabularly.
rowsOfMetrics = 2   
noOfModelsEvaluated = 5
metrics = np.zeros((rowsOfMetrics, noOfModelsEvaluated))


In [255]:
#Here we import and train and evaluate a Linear Regression model.
from sklearn.linear_model import LinearRegression

lrModel = LinearRegression()

lrKf = KFold(n_splits=2)

lrCvMseScore = cross_val_score(estimator=lrModel, X=X, y=y, cv=lrKf, scoring="neg_mean_squared_error")
lrCvR2Score = cross_val_score(estimator=lrModel, X=X, y=y, cv=lrKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 0] = np.mean(abs(lrCvMseScore))
metrics[1, 0] = np.mean(lrCvR2Score)

# print(lrCvMseScore)
# print(lrCvR2Score)
# print(np.mean(lrCvMseScore))
# print(np.mean(lrCvR2Score))

In [256]:

#Here we train and evaluate a Lasso Regession Model:
from sklearn.linear_model import Lasso

lassoModel = Lasso(alpha=0.05) #regularization strenth of 1.
lassoKf = KFold(n_splits=10)

lassoCvMseScore = cross_val_score(lassoModel, X, y, cv=lassoKf, scoring="neg_mean_squared_error")
lassoCvR2Score = cross_val_score(estimator=lassoModel, X=X, y=y, cv=lassoKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 1] = np.mean(abs(lassoCvMseScore))
metrics[1, 1] = np.mean(lassoCvR2Score)

In [257]:
# %%script false

#Here we train and evaluate a Ridge Regession Model:
from sklearn.linear_model import Ridge

ridgeModel = Ridge(alpha= 0.05)
ridgeKf = KFold(n_splits=10)

ridgeCvMseScore = cross_val_score(estimator=ridgeModel, X=X, y=y, cv=ridgeKf, scoring="neg_mean_squared_error")
ridgeCvRScore = cross_val_score(estimator=ridgeModel, X=X, y=y, cv=ridgeKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 2] = np.mean(abs(ridgeCvMseScore))
metrics[1, 2] = np.mean(ridgeCvRScore)


In [258]:
# %%script false
#Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rfRegressor = RandomForestRegressor(100, criterion='absolute_error', max_depth=4, random_state=42)
rfKfold = KFold(n_splits=10)

rfCvMseScore = cross_val_score(estimator=rfRegressor, X=X, y=y, cv=ridgeKf, scoring="neg_mean_squared_error")
rfCvR2Score = cross_val_score(estimator=rfRegressor, X=X, y=y, cv=ridgeKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 3] = np.mean(abs(rfCvMseScore))
metrics[1, 3] = np.mean(rfCvR2Score)


c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionW

In [259]:
# %%script false

#Training of an XGBoost model, to check if it gives a yet better peformance.
from xgboost import XGBRegressor
xgbModel = XGBRegressor(device="cpu",max_depth=4,random_state=42)
xgbKf = KFold(n_splits=10)

xgbCvMseScore = cross_val_score(estimator=xgbModel, X=X, y=y, cv=ridgeKf, scoring="neg_mean_squared_error")
xgbCvR2Score = cross_val_score(estimator=xgbModel, X=X, y=y, cv=ridgeKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 4] = np.mean(abs(xgbCvMseScore))
metrics[1, 4] = np.mean(xgbCvR2Score)

In [260]:
# %%script false

#Tabular visualization of performace metrics of models on trining and test data subsets.
index = ["MSE score", "R2 score"]
columns = ["OLSRegression", "LassoRegression", "RidgeRegression", "RandomForest", "XGBoost"]
metrics = pd.DataFrame(metrics, index=index, columns=columns)
metrics

,OLSRegression,LassoRegression,RidgeRegression,RandomForest,XGBoost
MSE score,2992.335630,2987.810600,2997.229190,3269.982650,4195.118823
R2 score,0.492439,0.465105,0.463474,0.414577,0.252512


Notes: 
1. As the alpha (regularization strength) value increases, the regularization increases causing the regularizeed models to underfit, performing more poorly than OLS.
2. MSE and r2 scores appear terible mainly due to smallness of dataset size.